<a href="https://colab.research.google.com/github/deva41103/StockGPT/blob/main/StockGptMl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This is a code for making models that can predict the  price for next trading session

In [ ]:
# Mounting drive for storage use
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Installing required dependencies
!pip install yfinance ta newsapi-python nltk tensorflow scikit-learn


  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=7f3dab9a51efa12f08c5f2015915870a9252a643772a410eb8ef1e3ade1554cf
  Stored in directory: /root/.cache/pip/wheels/5c/a1/5f/c6b85a7d9452057be4ce68a8e45d77ba34234a6d46581777c6
Successfully built ta


In [ ]:
# Importing required Libraries
import yfinance as yf
import numpy as np
import pandas as pd
import os
import json
from datetime import datetime

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError

import ta
from newsapi import NewsApiClient
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer


In [ ]:
# Download NLTK Sentiment Model
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
# CONFIGURATION
BASE_DIR = "/content/drive/MyDrive/NIFTY_LSTM_MODELS"
CACHE_DIR = f"{BASE_DIR}/sentiment_cache"

os.makedirs(BASE_DIR, exist_ok=True)
os.makedirs(CACHE_DIR, exist_ok=True)

START_DATE = "2018-01-01"
END_DATE = "2025-07-31"

LOOKBACK = 60
EPOCHS = 15
BATCH_SIZE = 32


In [ ]:
# News API Setup
NEWS_API_KEY = "ce263279b5154dafbb13b6df17d8abf1"

newsapi = NewsApiClient(api_key=NEWS_API_KEY)
sentiment_analyzer = SentimentIntensityAnalyzer()


In [ ]:
# Utility Functions - Dataset creator (multi-feature)
def create_dataset(data, lookback=60):
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[i:i + lookback])
        y.append(data[i + lookback, 0])  # Close price
    return np.array(X), np.array(y)


In [ ]:
# Progress tracker (auto-resume)
def load_progress(path):
    if os.path.exists(path):
        with open(path, "r") as f:
            return json.load(f)
    return {}

def save_progress(path, progress):
    with open(path, "w") as f:
        json.dump(progress, f)


In [ ]:
# WEEKLY SENTIMENT WITH CACHING
def get_weekly_sentiment(company, week_start):
    cache_file = f"{CACHE_DIR}/{company}.json"

    # Load cache
    if os.path.exists(cache_file):
        with open(cache_file, "r") as f:
            cache = json.load(f)
    else:
        cache = {}

    week_key = week_start.strftime("%Y-%m-%d")

    # If cached → return immediately
    if week_key in cache:
        return cache[week_key]

    # Otherwise call API
    try:
        articles = newsapi.get_everything(
            q=company,
            from_param=week_key,
            to=(week_start + pd.Timedelta(days=6)).strftime("%Y-%m-%d"),
            language="en",
            sort_by="relevancy",
            page_size=10
        )

        scores = []
        for a in articles["articles"]:
            text = (a["title"] or "") + " " + (a["description"] or "")
            scores.append(
                sentiment_analyzer.polarity_scores(text)["compound"]
            )

        sentiment = float(np.mean(scores)) if scores else 0.0

    except Exception:
        sentiment = 0.0

    # Save to cache
    cache[week_key] = sentiment
    with open(cache_file, "w") as f:
        json.dump(cache, f)

    return sentiment


In [ ]:
# TODAY’S 5-STOCK BATCH
TODAY_STOCKS = [
    "ADANIENT.NS",
    "ADANIPORTS.NS",
    "APOLLOHOSP.NS",
    "ASIANPAINT.NS",
    "AXISBANK.NS",
    "BAJAJ-AUTO.NS",
    "BAJFINANCE.NS",
    "BAJAJFINSV.NS",
    "BPCL.NS",
    "BHARTIARTL.NS",
    "BRITANNIA.NS",
    "CIPLA.NS",
    "COALINDIA.NS",
    "DIVISLAB.NS",
    "DRREDDY.NS",
    "EICHERMOT.NS",
    "GRASIM.NS",
    "HCLTECH.NS",
    "HDFCLIFE.NS",
    "HEROMOTOCO.NS",
    "HINDALCO.NS",
    "HINDUNILVR.NS",
    "ITC.NS",
    "JSWSTEEL.NS",
    "KOTAKBANK.NS",
    "LT.NS",
    "LTIM.NS",
    "MARUTI.NS",
    "NESTLEIND.NS",
    "NTPC.NS",
    "ONGC.NS",
    "POWERGRID.NS",
    "SBILIFE.NS",
    "SBIN.NS",
    "SUNPHARMA.NS",
    "TATACONSUM.NS",
    "TATAMOTORS.NS",
    "TATASTEEL.NS",
    "TECHM.NS",
    "TITAN.NS",
    "ULTRACEMCO.NS",
    "UPL.NS",
    "WIPRO.NS"
]

print("📊 Training batch:", TODAY_STOCKS)


📊 Training batch: ['ADANIENT.NS', 'ADANIPORTS.NS', 'APOLLOHOSP.NS', 'ASIANPAINT.NS', 'AXISBANK.NS', 'BAJAJ-AUTO.NS', 'BAJFINANCE.NS', 'BAJAJFINSV.NS', 'BPCL.NS', 'BHARTIARTL.NS', 'BRITANNIA.NS', 'CIPLA.NS', 'COALINDIA.NS', 'DIVISLAB.NS', 'DRREDDY.NS', 'EICHERMOT.NS', 'GRASIM.NS', 'HCLTECH.NS', 'HDFCLIFE.NS', 'HEROMOTOCO.NS', 'HINDALCO.NS', 'HINDUNILVR.NS', 'ITC.NS', 'JSWSTEEL.NS', 'KOTAKBANK.NS', 'LT.NS', 'LTIM.NS', 'MARUTI.NS', 'NESTLEIND.NS', 'NTPC.NS', 'ONGC.NS', 'POWERGRID.NS', 'SBILIFE.NS', 'SBIN.NS', 'SUNPHARMA.NS', 'TATACONSUM.NS', 'TATAMOTORS.NS', 'TATASTEEL.NS', 'TECHM.NS', 'TITAN.NS', 'ULTRACEMCO.NS', 'UPL.NS', 'WIPRO.NS']


In [ ]:
import os
import json
import joblib
import numpy as np
import pandas as pd
import yfinance as yf
import ta

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError

# ---------------- CONFIG ---------------- #
LOOKBACK = 60
EPOCHS = 15
BATCH_SIZE = 32

# ---------------- DATASET CREATOR ---------------- #
def create_dataset(data, lookback=60):
    X, y = [], []
    for i in range(len(data) - lookback - 1):
        X.append(data[i:i + lookback])
        y.append(data[i + lookback + 1, 0])  # next-day Close
    return np.array(X), np.array(y)

# ---------------- TRAIN LOOP ---------------- #
for symbol in TODAY_STOCKS:
    print(f"\n🚀 Processing {symbol}")

    stock_dir = f"{BASE_DIR}/{symbol}"
    os.makedirs(stock_dir, exist_ok=True)

    model_path = f"{stock_dir}/lstm_sentiment.h5"
    scaler_path = f"{stock_dir}/scaler.pkl"
    progress_path = f"{stock_dir}/progress.json"

    # Load progress
    last_epoch = 0
    if os.path.exists(progress_path):
        with open(progress_path, "r") as f:
            last_epoch = json.load(f).get("epoch", 0)

    # -------- DOWNLOAD DATA (FIXED) -------- #
    df = yf.download(
        symbol,
        start=START_DATE,
        end=END_DATE,
        auto_adjust=False  # 🔴 VERY IMPORTANT
    )

    if df.empty:
        print("❌ No data, skipping")
        continue

    df = df[['Close', 'Volume']]

    # -------- INDICATORS -------- #
    close = df['Close'].squeeze()

    df['RSI'] = ta.momentum.RSIIndicator(close, window=14).rsi()

    macd = ta.trend.MACD(close)
    df['MACD'] = macd.macd()
    df['MACD_signal'] = macd.macd_signal()

    # -------- SENTIMENT -------- #
    company = symbol.replace(".NS", "")
    sentiments = []
    for date in df.index:
        week_start = date - pd.Timedelta(days=date.weekday())
        sentiments.append(get_weekly_sentiment(company, week_start))

    df['Sentiment'] = sentiments
    df.dropna(inplace=True)

    # -------- FEATURES -------- #
    features = ['Close', 'Volume', 'RSI', 'MACD', 'MACD_signal', 'Sentiment']

    # -------- SCALING (SAVE SCALER) -------- #
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(df[features])
    joblib.dump(scaler, scaler_path)

    # -------- DATASET -------- #
    X, y = create_dataset(scaled, LOOKBACK)

    # -------- MODEL -------- #
    if os.path.exists(model_path):
        model = load_model(model_path, custom_objects={"mse": MeanSquaredError()})
        print(f"🔁 Resuming from epoch {last_epoch}")
    else:
        model = Sequential([
            LSTM(64, return_sequences=True, input_shape=(LOOKBACK, X.shape[2])),
            Dropout(0.2),
            LSTM(64),
            Dropout(0.2),
            Dense(1)
        ])
        model.compile(optimizer="adam", loss="mse")
        print("🆕 Starting new model")

    checkpoint = ModelCheckpoint(
        model_path,
        save_best_only=False,
        save_weights_only=False,
        verbose=0
    )

    # -------- TRAIN -------- #
    model.fit(
        X, y,
        epochs=EPOCHS,
        initial_epoch=last_epoch,
        batch_size=BATCH_SIZE,
        callbacks=[checkpoint],
        verbose=1
    )

    # -------- SAVE PROGRESS -------- #
    with open(progress_path, "w") as f:
        json.dump({"epoch": EPOCHS}, f)

    print(f"✅ Completed {symbol}")



🚀 Processing ADANIENT.NS


[*********************100%***********************]  1 of 1 completed


🆕 Starting new model
Epoch 1/15


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


53/56 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0612

...

56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0026
Epoch 12/15
53/56 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0020

56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0021
Epoch 13/15
53/56 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0018

56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0019
Epoch 14/15
52/56 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0020

56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0020
Epoch 15/15
52/56 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0019

56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0019
✅ Completed WIPRO.NS


In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import ta
import joblib

from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanSquaredError

# ================= CONFIG ================= #
symbol = "SUNPHARMA.NS"

# ================= LOAD DATA ================= #
df = yf.download(
    symbol,
    start=START_DATE,
    end=END_DATE,
    auto_adjust=False
)

df = df[['Close', 'Volume']]

# ================= FIX CLOSE (FINAL & CORRECT) ================= #
close = pd.Series(
    df['Close'].to_numpy().reshape(-1),
    index=df.index
)

# ================= INDICATORS ================= #
df['RSI'] = ta.momentum.RSIIndicator(close=close, window=14).rsi()

macd = ta.trend.MACD(close=close)
df['MACD'] = macd.macd()
df['MACD_signal'] = macd.macd_signal()

df.dropna(inplace=True)

# ================= SENTIMENT ================= #
company = symbol.replace(".NS", "")
df['Sentiment'] = [
    get_weekly_sentiment(company, d - pd.Timedelta(days=d.weekday()))
    for d in df.index
]

df.dropna(inplace=True)

# ================= FEATURES ================= #
features = ['Close', 'Volume', 'RSI', 'MACD', 'MACD_signal', 'Sentiment']

# ================= LOAD TRAINING SCALER ================= #
scaler = joblib.load(f"{BASE_DIR}/{symbol}/scaler.pkl")

scaled = scaler.transform(df[features])
last_60 = scaled[-LOOKBACK:].reshape(1, LOOKBACK, scaled.shape[1])

# ================= LOAD MODEL ================= #
model = load_model(
    f"{BASE_DIR}/{symbol}/lstm_sentiment.h5",
    custom_objects={"mse": MeanSquaredError()}
)

# ================= MC DROPOUT ================= #
def mc_dropout_predict(model, x, n_samples=100):
    preds = []
    for _ in range(n_samples):
        preds.append(float(model(x, training=True).numpy()[0][0]))
    return np.array(preds)

mc_preds = mc_dropout_predict(model, last_60)

mean_scaled = float(mc_preds.mean())
std_scaled = float(mc_preds.std())

# ================= INVERSE TRANSFORM ================= #
def inverse_close(scaled_close):
    dummy = np.zeros((1, scaled.shape[1]))
    dummy[0, 0] = scaled_close
    return float(scaler.inverse_transform(dummy)[0, 0])

mean_price = inverse_close(mean_scaled)
upper_price = inverse_close(mean_scaled + std_scaled)
lower_price = inverse_close(mean_scaled - std_scaled)

# ================= LAST CLOSE ================= #
last_close = float(df['Close'].iloc[-1])

# ================= DIRECTION ================= #
direction = "UP 📈" if mean_price > last_close else "DOWN 📉"

# ================= CONFIDENCE ================= #
confidence = 0.0
if abs(mean_scaled) > 1e-8:
    confidence = 100 * (1 - std_scaled / abs(mean_scaled))
confidence = float(np.clip(confidence, 0, 100))

# ================= MULTI-HORIZON FORECAST ================= #
def multi_horizon_forecast(model, window, steps):
    temp = window.copy()
    preds = []

    for _ in range(steps):
        pred = float(model(temp, training=False).numpy()[0][0])
        preds.append(pred)

        next_row = temp[0, -1].copy()
        next_row[0] = pred
        temp = np.roll(temp, -1, axis=1)
        temp[0, -1] = next_row

    return preds

forecast_1d = mean_price
forecast_1m = inverse_close(np.mean(multi_horizon_forecast(model, last_60, 30)))
forecast_1y = inverse_close(np.mean(multi_horizon_forecast(model, last_60, 252)))

# ================= OUTPUT ================= #
print("\n📊 STOCK FORECAST SUMMARY")
print("-----------------------------------")
print(f"Symbol            : {symbol}")
print(f"Last Close        : ₹{last_close:.2f}")
print(f"Expected Price    : ₹{mean_price:.2f}")
print(f"Likely Range      : ₹{lower_price:.2f} – ₹{upper_price:.2f}")
print(f"Direction         : {direction}")
print(f"Confidence        : {confidence:.1f}%")

print("\n⏳ HORIZON-BASED FORECAST (VISION)")
print("-----------------------------------")
print(f"1 Day Forecast    : ₹{forecast_1d:.2f}")
print(f"1 Month Forecast  : ₹{forecast_1m:.2f}")
print(f"1 Year Forecast   : ₹{forecast_1y:.2f}")


[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-1272433734.py:85: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  last_close = float(df['Close'].iloc[-1])



📊 STOCK FORECAST SUMMARY
-----------------------------------
Symbol            : SUNPHARMA.NS
Last Close        : ₹1733.80
Expected Price    : ₹1760.01
Likely Range      : ₹1661.21 – ₹1858.81
Direction         : UP 📈
Confidence        : 93.1%

⏳ HORIZON-BASED FORECAST (VISION)
-----------------------------------
1 Day Forecast    : ₹1760.01
1 Month Forecast  : ₹1886.99
1 Year Forecast   : ₹2231.73
